# Installing Dependecies

In [1]:
!pip install pandas
!pip install networkx
!pip install matplotlib
!pip install scipy
!pip install dash_bootstrap_components
!pip install dash dash-core-components dash-html-components dash-cytoscape networkx

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [20]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import random

In [21]:
def load_adjacency_list(csv_file):
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(csv_file)
    # Convert the DataFrame to a list of tuples
    edges = [tuple(x) for x in df.values]
    return edges

In [22]:
def create_graph_from_adj_list(edges):
    # Create a NetworkX graph from the adjacency list
    G = nx.Graph()
    G.add_edges_from(edges)
    return G

In [23]:
csv_file = "roads-usroads/edges.csv"
edges = load_adjacency_list(csv_file)
G = create_graph_from_adj_list(edges)

In [24]:
print(f"Number of nodes: {G.number_of_nodes()}")
print(f"Number of edges: {G.number_of_edges()}")

Number of nodes: 129163
Number of edges: 165434


In [25]:
# Function to get the largest connected component
def get_largest_connected_component(G):
    largest_cc = max(nx.connected_components(G), key=len)
    largest_subgraph = G.subgraph(largest_cc)
    return largest_subgraph

In [26]:
# Function to sample nodes from the largest connected component
def sample_largest_connected_component(largest_subgraph, node_sample_size):
    nodes_list = list(largest_subgraph.nodes())
    if len(nodes_list) > node_sample_size:
        sampled_nodes = random.sample(nodes_list, node_sample_size)
        subgraph_sample = largest_subgraph.subgraph(sampled_nodes)
    else:
        subgraph_sample = largest_subgraph

    if not nx.is_connected(subgraph_sample):
        largest_sample_cc = max(nx.connected_components(subgraph_sample), key=len)
        subgraph_sample = subgraph_sample.subgraph(largest_sample_cc)

    return subgraph_sample

In [27]:
# Function to calculate shortest path lengths and return all-pair shortest paths
def get_shortest_paths(subgraph_sample):
    path_lengths = dict(nx.shortest_path_length(subgraph_sample))
    shortest_paths = []

    for node, lengths in path_lengths.items():
        shortest_paths.extend(list(lengths.values()))
    
    return shortest_paths

In [28]:
# Function to calculate graph metrics (diameter, average path length, and shortest path)
def calculate_graph_metrics(subgraph_sample):
    if nx.is_connected(subgraph_sample):
        diameter = nx.diameter(subgraph_sample)
        avg_path_length = nx.average_shortest_path_length(subgraph_sample)
        node1, node2 = random.sample(list(subgraph_sample.nodes()), 2)
        shortest_path_length = nx.shortest_path_length(subgraph_sample, source=node1, target=node2)
        print(f"Diameter of the subgraph: {diameter}")
        print(f"Average Path Length of the subgraph: {avg_path_length}")
        print(f"Shortest path between {node1} and {node2}: {shortest_path_length}")
    else:
        print("The sampled subgraph is not connected.")

In [29]:
# Get the largest connected component
largest_subgraph = get_largest_connected_component(G)
    
# Sample the largest connected component
subgraph_sample = sample_largest_connected_component(largest_subgraph, 10000)

In [27]:
calculate_graph_metrics(subgraph_sample)

Diameter of the subgraph: 610
Average Path Length of the subgraph: 225.2216546381362
Shortest path between 22814 and 51840: 368


In [30]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import dash_cytoscape as cyto
import networkx as nx
import random

# Create a random graph using NetworkX
def generate_graph(node_sample_size=100):
    # Generate a random graph
    G = nx.erdos_renyi_graph(500, 0.05)
    largest_cc = max(nx.connected_components(G), key=len)
    largest_subgraph = G.subgraph(largest_cc)
    
    nodes_list = list(largest_subgraph.nodes())
    
    if len(nodes_list) > node_sample_size:
        sampled_nodes = random.sample(nodes_list, node_sample_size)
        sampled_subgraph = largest_subgraph.subgraph(sampled_nodes)
    else:
        sampled_subgraph = largest_subgraph

    elements = []
    
    # Create nodes and edges for Cytoscape
    for node in sampled_subgraph.nodes:
        elements.append({
            'data': {'id': str(node), 'label': f'Node {node}'},
            'position': {'x': random.randint(0, 800), 'y': random.randint(0, 800)},
            'classes': 'node'
        })
    for edge in sampled_subgraph.edges:
        elements.append({'data': {'source': str(edge[0]), 'target': str(edge[1])}})
    
    return elements

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the app layout
app.layout = html.Div([
    html.H1("Dynamic Graph Dashboard"),
    
    # Slider for selecting the number of nodes
    html.Div([
        html.Label('Select number of nodes:'),
        dcc.Slider(
            id='node-slider',
            min=10,
            max=500,
            step=10,
            value=100,
            marks={i: f'{i}' for i in range(10, 501, 50)}
        )
    ], style={'width': '50%', 'margin': '20px'}),
    
    # Cytoscape graph component
    cyto.Cytoscape(
        id='network-graph',
        layout={'name': 'preset'},  # 'preset' lets you use random positioning
        style={'width': '100%', 'height': '600px'},
        elements=generate_graph(),
        stylesheet=[
            {
                'selector': 'node',
                'style': {
                    'background-color': '#333333',
                    'label': 'data(label)',
                    'font-size': '10px',
                    'color': '#FF0000'  # Red color for node labels
                }
            },
            {
                'selector': 'edge',
                'style': {
                    'line-color': '#555555',
                    'width': 2
                }
            }
        ]
    )
])

# Define the callback to update the graph based on the slider input
@app.callback(
    Output('network-graph', 'elements'),
    [Input('node-slider', 'value')]
)
def update_graph(node_sample_size):
    return generate_graph(node_sample_size)

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

In [13]:
def k_shortest_paths(subgraph_sample, k):
    # Randomly choose source and destination from subgraph
    nodes = list(subgraph_sample.nodes())
    source, target = random.sample(nodes, 2)  # Pick two distinct nodes
    
    try:
        paths = list(nx.shortest_simple_paths(subgraph_sample, source, target))[:k]
        return source, target, paths
    except nx.NetworkXNoPath:
        return source, target, []

In [15]:
# Get random k-shortest paths
source, target, k_paths = k_shortest_paths(subgraph_sample, 5)

print(f"Source: {source}, Target: {target}")
print(f"K-Shortest Paths: {k_paths}")

Source: 22797, Target: 22329
K-Shortest Paths: [[np.int64(22797), np.int64(22830), np.int64(22329)]]
